In [8]:
#from xml.dom import minidom
#import collections

import os
import cv2
import random

import matplotlib.pyplot as plt
import matplotlib as matplot
#import seaborn as sns
#%matplotlib inline
import json



import yaml

In [ ]:
# Read YAML file
with open("config.yaml", 'r') as stream:
    config_file = yaml.safe_load(stream)

DATASET_SRC_DIR = config_file['dataset_src_dir']
#DATASET_SRC_DIR = os.getcwd()
imageSetsPath = os.path.join(os.getcwd(), 'ImageSets/')


# Read YAML file
with open("coco_hurtigruten.json", 'r') as stream:
    data_loaded = yaml.safe_load(stream)
    
CLASSES = list(data_loaded['names'].values())
NEW_DATASET_DIR = data_loaded['path'][1:]
VAL_PATH = os.path.join(NEW_DATASET_DIR, data_loaded['val'])
TRAIN_PATH = os.path.join(NEW_DATASET_DIR, data_loaded['train'])
classes_dict = {"D00": "Longitude cracks", "D10": "Lateral", "D20": "Alligator", "D40": "Pothole"}

In [9]:
ann_file = "./hurtigruten/coco_hurtigruten.json"
f = open(ann_file)
anns = json.load(f)
CLASSES = {elm['id'] : elm['name'] for elm in anns['categories']}

In [12]:
numb_images = ans['ima']

In [2]:
!conda install pycocotools

Solving environment: done

# All requested packages already installed.



In [4]:
from conda pycocotools.coco import COCO

SyntaxError: invalid syntax (3359594458.py, line 1)

In [ ]:
ann_file = "./hurtigruten/coco_hurtigruten.json"

coco=COCO(ann_file)

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from PIL import Image
import requests
from pycocotools.coco import COCO


coco_annotation_file_path = "./hurtigruten/coco_hurtigruten.json"

coco_annotation = COCO(annotation_file=coco_annotation_file_path)

In [ ]:
# Category IDs.
cat_ids = coco_annotation.getCatIds()
print(f"Number of Unique Categories: {len(cat_ids)}")
print("Category IDs:")
print(cat_ids)  # The IDs are not necessarily consecutive.

# All categories.
cats = coco_annotation.loadCats(cat_ids)
cat_names = [cat["name"] for cat in cats]
print("Categories Names:")
print(cat_names)

In [ ]:
# Category ID -> Category Name.
query_id = cat_ids[0]
query_annotation = coco_annotation.loadCats([query_id])[0]
query_name = query_annotation["name"]
query_supercategory = query_annotation["supercategory"]
print("Category ID -> Category Name:")
print(
    f"Category ID: {query_id}, Category Name: {query_name}, Supercategory: {query_supercategory}"
)

In [ ]:
# Category Name -> Category ID.
query_name = cat_names[2]
query_id = coco_annotation.getCatIds(catNms=[query_name])[0]
print("Category Name -> ID:")
print(f"Category Name: {query_name}, Category ID: {query_id}")

In [ ]:
# Get the ID of all the images containing the object of the category.
img_ids = coco_annotation.getImgIds(catIds=[query_id])
print(f"Number of Images Containing {query_name}: {len(img_ids)}")

In [ ]:
# Pick one image.
img_id = img_ids[2]
img_info = coco_annotation.loadImgs([img_id])[0]
img_file_name = img_info["file_name"]
img_url = img_info["coco_url"]
print(
    f"Image ID: {img_id}, File Name: {img_file_name}, Image URL: {img_url}"
)

# Get all the annotations for the specified image.
ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
anns = coco_annotation.loadAnns(ann_ids)
print(f"Annotations for Image ID {img_id}:")
print(anns)

# Use URL to load image.
im = Image.open(requests.get(img_url, stream=True).raw)

# Save image and its labeled version.
plt.axis("off")
plt.imshow(np.asarray(im))
plt.savefig(f"{img_id}.jpg", bbox_inches="tight", pad_inches=0)
# Plot segmentation and bounding box.
coco_annotation.showAnns(anns, draw_bbox=True)
plt.savefig(f"{img_id}_annotated.jpg", bbox_inches="tight", pad_inches=0)